In [1]:
!pip install gym

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pwd

/content


In [7]:
!cp -r drive/MyDrive/gym_examples /content

In [9]:
!ls gym_examples

envs  __init__.py


In [10]:
import gym
env = gym.make('gym_examples:gym_examples/ShootingAirplane-v0', render_mode='text')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [11]:
obs, info = env.reset()

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'tuple'>
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:226: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:167: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space with exception: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
  logger.warn(f"{pre} is not with

In [12]:
env.observation_space

Box(0, 255, (8, 8, 1), uint8)

In [13]:
obs.shape

(8, 8, 1)

In [14]:
env.render()

         |         
         |         
         |   H     
         | HHHHH   
         |   H     
         |  HHH    
         |         
         |         



In [ ]:
obs, reward, done,info = env.step((0, 0))

In [ ]:
env.render()

M        |         
         |     HHH 
         |      H  
         |    HHHHH
         |      H  
         |         
         |         
         |         



In [15]:
obs, reward, done,info = env.step((4, 4))

In [16]:
env.render()

         |         
         |         
         |   H     
         | HHHHH   
    M    |   H     
         |  HHH    
         |         
         |         



In [17]:
obs, reward, done,info = env.step((3, 3))
env.render()

         |         
         |         
         |   H     
   H     | HHHHH   
    M    |   H     
         |  HHH    
         |         
         |         



In [18]:
reward  # Hit일 때의 reward는 1

1

In [19]:
obs, reward, done,info = env.step((1, 3))
env.render()

         |         
   M     |         
         |   H     
   H     | HHHHH   
    M    |   H     
         |  HHH    
         |         
         |         



In [20]:
reward  # Miss일 때의 reward는 -1

-1

# DQN

In [21]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T

In [22]:
gamma = 0.99
epsilon = 1.0
epsilon_max = 1.0
epsilon_min = 0.1
epsilon_interval = epsilon_max - epsilon_min
batch_size = 16
max_steps_per_episode = 60
max_episodes = 10000

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
env.observation_space

Box(0, 255, (8, 8, 1), uint8)

In [24]:
env.action_space

MultiDiscrete([8 8])

In [25]:
num_actions = 64

In [26]:
class QModel(nn.Module):
  def __init__(self, num_actions):
    super(QModel, self).__init__()
    self.dropout = nn.Dropout(p=0.3)
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding='same')
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same')
    self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(1152, 512)
    self.fc2 = nn.Linear(512, num_actions)

  def forward(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = nn.functional.relu(self.conv2(x))
    x = self.dropout(x)
    x = nn.functional.relu(self.conv3(x))
    x = self.flatten(x)
    x = nn.functional.relu(self.fc1(x))
    x = self.dropout(x)
    return self.fc2(x)

In [27]:
model = QModel(num_actions)

In [28]:
model_target = QModel(num_actions)

In [29]:
loss_function = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

In [30]:
action_history = []
action_mask_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []

In [31]:
episode_reward_history = []
running_reward = 0.
episode_count = 0
frame_count = 0

In [32]:
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 200000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 500000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000

In [33]:
def preprocess_state(obs):
  st = torch.from_numpy(obs).squeeze()
  st = st.to(torch.int64)
  st = torch.nn.functional.one_hot(st, num_classes=3)
  st = st.permute(2, 0, 1)
  return st.to(torch.float32)

In [34]:
board, info = env.reset()

In [35]:
board.shape

(8, 8, 1)

In [36]:
st = preprocess_state(board)
st.shape

torch.Size([3, 8, 8])

In [37]:
st

tensor([[[1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
       

In [38]:
def get_greedy_epsilon(model, state, mask):
  global epsilon

  #if frame_count < epsilon_random_frames or np.random.rand(1)[0] < epsilon:
  if np.random.rand(1)[0] < epsilon:
    action = np.random.choice([ i for i in range(num_actions) if mask[i] == 1 ])
  else:
    with torch.no_grad():
      # add a batch axis
      state_tensor = state.unsqueeze(0)
      # compute the q-values
      q_values = model(state_tensor)
      # select the q-values of valid actions
      action = torch.argmax(
        q_values.squeeze() + torch.from_numpy(mask) * 100., dim=0)

  epsilon -= epsilon_interval / epsilon_greedy_frames
  epsilon = max(epsilon, epsilon_min)
  return action

In [39]:
def get_greedy_action(model, state, mask):
  global epsilon
  with torch.no_grad():
    state_tensor = state.unsqueeze(0) # batch dimension
    q_values = model(state_tensor)
    action = torch.argmax(
      q_values.squeeze() + torch.from_numpy(mask) * 100.,dim=0)
  return action

In [40]:
def sample_batch(_batch_size):
  indices = np.random.choice(range(len(done_history)), size=_batch_size, replace=False)
  state_sample = np.array([state_history[i].squeeze(0).numpy() for i in indices])
  state_next_sample = np.array([state_next_history[i].squeeze(0).numpy() for i in indices])
  rewards_sample = np.array([rewards_history[i] for i in indices], dtype=np.float32)
  action_sample = np.array([action_history[i] for i in indices])

  # action mask is the mask for the valid actions at the '''next''' state
  action_mask_sample = np.array([action_mask_history[i] for i in indices])
  done_sample = np.array([float(done_history[i]) for i in indices])
  return state_sample, state_next_sample, rewards_sample, action_sample, action_mask_sample, done_sample

In [41]:
def update_network():
  state_sample, state_next_sample, rewards_sample, action_sample, action_mask_sample, done_sample = \
    sample_batch(batch_size)

  state_sample = torch.tensor(state_sample, dtype=torch.float32)
  state_next_sample = torch.tensor(state_next_sample, dtype=torch.float32)
  action_sample = torch.tensor(action_sample, dtype=torch.int64)
  action_mask_sample = torch.tensor(action_mask_sample, dtype=torch.int64)
  rewards_sample = torch.tensor(rewards_sample, dtype=torch.float32)
  done_sample = torch.tensor(done_sample, dtype=torch.float32)

  with torch.no_grad():
    future_rewards = model_target(state_next_sample)
    max_q_values = torch.max (
        future_rewards + action_mask_sample * 100.,
    dim=1).values.detach() - 100.
    target_q_values = rewards_sample + gamma * max_q_values * (1. - done_sample)

  q_values = model(state_sample)
  q_values_action = q_values.gather(dim=1, index=action_sample.unsqueeze(1)).squeeze(1)
  loss = loss_function(q_values_action, target_q_values)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


In [42]:
for _ in range(max_episodes):
  state, info = env.reset()
  state = preprocess_state(state)
  action_mask = info['action_mask'].reshape((-1,))
  episode_reward = 0

  for timestep in range(1, max_steps_per_episode):
    frame_count += 1

    action = get_greedy_epsilon(model, state, action_mask)

    state_next, reward, done, info = env.step((action // 8, action % 8))
    state_next = preprocess_state(state_next)
    action_mask = info['action_mask'].reshape((-1,))

    episode_reward += reward

    action_history.append(action)
    action_mask_history.append(action_mask)
    state_history.append(state)
    state_next_history.append(state_next)
    rewards_history.append(reward)
    done_history.append(done)

    state = state_next

    if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
      update_network()

    if frame_count % update_target_network == 0:
      model_target.load_state_dict(model.state_dict())

    if len(rewards_history) > max_memory_length:
      del rewards_history[:1]
      del state_history[:1]
      del state_next_history[:1]
      del action_history[:1]
      del action_mask_history[:1]
      del done_history[:1]
    if done:
      break

  episode_count +=1
  episode_reward_history.append(episode_reward)

  if len(episode_reward_history)> 100:
    del episode_reward_history[0]

  running_reward = np.mean(episode_reward_history)

  if episode_count % 10 == 0:
    print(episode_count, frame_count, running_reward)

10 549 -35.9
20 1127 -37.65
30 1707 -38.166666666666664
40 2265 -37.925
50 2838 -38.16
60 3411 -38.21666666666667
70 3997 -38.55714285714286
80 4563 -38.5375
90 5145 -38.67777777777778
100 5711 -38.57
110 6288 -38.91
120 6853 -38.78
130 7437 -38.88
140 8017 -39.1
150 8577 -38.87
160 9118 -38.53
170 9694 -38.33
180 10244 -38.09
190 10778 -37.57
200 11347 -37.64
210 11916 -37.46
220 12469 -37.26
230 13030 -36.95
240 13571 -36.48
250 14114 -36.35
260 14652 -36.28
270 15220 -36.2
280 15797 -36.51
290 16310 -36.24
300 16839 -35.76
310 17379 -35.55
320 17908 -35.27
330 18418 -34.68
340 18936 -34.41
350 19456 -34.14
360 19984 -33.98
370 20518 -33.56
380 21074 -33.33
390 21612 -33.58
400 22157 -33.72
410 22670 -33.33
420 23208 -33.46
430 23746 -33.84
440 24248 -33.68
450 24722 -33.14
460 25227 -32.93
470 25713 -32.47
480 26206 -31.74
490 26716 -31.46
500 27169 -30.54
510 27685 -30.59
520 28192 -30.22
530 28673 -29.53
540 29151 -29.27
550 29640 -29.42
560 30132 -29.29
570 30613 -29.22
580 31104

In [43]:
import time, sys
from IPython.display import clear_output
board, info = env.reset()
state = preprocess_state(board)
action_mask = info['action_mask'].reshape((-1,))
done = False
env.render()
while not done:
  action = get_greedy_action(model, state, action_mask)
  print("action: ({}, {})".format(action // 8, action % 8))
  sys.stdout.flush()
  time.sleep(1.0)
  clear_output(wait=False)
  board, reward, done, info = env.step((action // 8, action % 8))
  state = preprocess_state(board)
  action_mask = info['action_mask'].reshape((-1,))
  env.render()


         |         
         |         
     M   |         
  M H    |     H   
  M HMH  |     H H 
   HHHH  |    HHHH 
    H H  |     H H 
    H    |     H   

